This notebook is meant to replace 4-3-UserStudy2_preference_hierarchical since the code is cleaner and easier to debug. The old version also has some bugs in the hMatch calculation.

In [9]:
import os
import re
import itertools
import pandas as pd

from bert_score import BERTScorer
from collections import defaultdict

from src.config import substitution_rating_file, preference_rating_file
from src.config import preference_rating_scores
from src.config import label_file, label_file_full, weighing_scheme
from src.config import lst_ref
from src.io import load_pickle, save_pickle

import warnings
warnings.filterwarnings('ignore')

In [10]:
import time
start = time.time()
lst_ref = {v: k.split('__')[-1].replace('_', ' ') for k, v in lst_ref.items()}
end = time.time()
print(end - start)

0.0009984970092773438


In [11]:
def process_food_name(s1):
    # separators: ", " + any of (integer, decimal & fraction) +" "
    exp = r", \d+\.\d+ |, \d+\,\d+ |, \d+ |, \d+\/\d+ "
    # remove content in parenthesis for finding the separator
    if s1.count('(') == s1.count(')'):
        s2 = re.sub(r'[(].*?[\)]', ' ', s1)
    else:
        s2 = s1
    try:
        split_by = re.findall(exp, s2)[0]
        return clean_name(s1.split(split_by)[0])
    except:
        return clean_name(s2)

def clean_name(name):
    name = name.replace("\t", " ").replace("\n", " ").replace("w/o", " no ").replace("w/", " ")
    return re.sub(' +', ' ', name.strip()).lower()

In [17]:
import time
start = time.time()

# ground truth
df2 = pd.read_csv(substitution_rating_file)[['user', 'item_1']].drop_duplicates().reset_index(drop=True)
df2 = df2[['user', 'item_1']]
df2['item_gt'] = df2['item_1'].apply(process_food_name)

# predictions
df = pd.read_csv(preference_rating_file)
df['item'] = df['choice'].apply(process_food_name)
gt = df2.groupby('user')['item_gt'].apply(list).to_dict()
df['gt'] = df['user'].map(gt)
end = time.time()
print(end - start)

0.031914710998535156


In [13]:
labels = load_pickle(label_file)
concat_list = list(zip(labels['cat_info'], labels['label_summary']))
label_index = dict()
for i,j in concat_list:
    label_index.update(zip(i,j))  

index_label = {k:v for v,k in label_index.items()}

# '_' connected tokens for l0 tags
label_index_l0 = {i.split('__')[-1]:v for i,v in label_index.items() if len(i.split('__'))==3}
l0_tags = sorted(label_index_l0.keys())

label_with_food = defaultdict(list)
def group_food_name(line):
    s = line['food_name']
    for i in line['label_summary']:
        label_with_food[i].append(s)
labels.apply(group_food_name, axis=1) 

label_name_l0 = {}
for l,i in label_index_l0.items():
    label_name_l0[l] = label_with_food[i]

name_labels = labels.set_index('food_name').to_dict()['cat_info']
name_label_0 = {k:[s.split('__')[-1] for s in v if len(s.split('__'))==3] for k,v in name_labels.items()}

matched_label_l0 = []
unmatched_label_l0 = []

def token_transform(t):
    tokens = [t, t+'s', t+'es']
    if t[-1] == 'y':
        tokens.append(t[:-1]+'ies')
    return tokens  

for l, s_lst in label_name_l0.items():    
    l_primes = [token_transform(t) for t in l.split('_')]
    l2 = list(itertools.product(*l_primes))
    l_primes = [' '.join(i) for i in l2]  
 
    matched = False
    for s in s_lst:
        s = ' '.join(s.replace("'", '').replace('&', ' ').split())
        for l_prime in l_primes:
            if l_prime in s:            
                matched = True
            
    if matched:
        matched_label_l0.append(l)
    else:
        unmatched_label_l0.append(l)  

perc = len(unmatched_label_l0)/(len(matched_label_l0 )+ len(unmatched_label_l0)) 
if perc < 0.05:
    print('Less than 5% labels are not matched: {:.2%}'.format(perc))

# get all labels associated with item
def match_labels(s):
    if s in name_label_0.keys():
        found = name_label_0[s] 
    else:
        found = []
        s = ' '.join(s.replace("'", '').replace('&', ' ').split())
        for l in matched_label_l0:
            matched = False
            l_primes = [token_transform(t) for t in l.split('_')]
            for l_prime in [' '.join(i) for i in list(itertools.product(*l_primes))]:
                if l_prime in s:  
                    matched = True
            if matched:
                found.append(l) 
                        
    # full label names
    all_labels = []
    if len(found) > 0:
        full_label_l0 = [index_label[label_index_l0[l0_label]] for l0_label in found]
        for l0 in full_label_l0:
            l2 = l0.split('__')[0]
            l1 = '__'.join(l0.split('__')[:2])
            all_labels.extend([l2, l1, l0])
    return sorted(set(all_labels))

Less than 5% labels are not matched: 1.33%


In [14]:
df.loc[:, 'label']  = df['item'].apply(match_labels)
df['label_summary'] = df['label'].apply(lambda s: [label_index[i] for i in s] if len(s)>0 else [])

df2.loc[:, 'label']  = df2['item_gt'].apply(match_labels)
df2['label_summary'] = df2['label'].apply(lambda s: [label_index[i] for i in s] if len(s)>0 else [])
df2 = df2[['user', 'item_1', 'label', 'label_summary']]

gt_lst = df2.groupby('user')['label_summary'].apply(list).to_dict()
df['l2'] = df['user'].map(gt_lst)

df1 = df.groupby(['user', 'qn'])['label_summary'].apply(list).reset_index().rename(columns={'label_summary': 'l2'})
df1 = df2.merge(df1, on='user', how='outer')

In [8]:
labels = load_pickle(label_file_full)
lst = sorted(set([item for sublist in labels['labels'].tolist() for item in sublist]))
lst_ref = {v: k.split('__')[-1].replace('_', ' ') for v, k in enumerate(lst)}

In [10]:
def label_score(c_cap_j, c_cap_j_prime, weight):
    if len(c_cap_j)==0 or len(c_cap_j_prime)==0:
        return 0
    
    numerator = 0.0
    denominator = 0.0

    for c_t in c_cap_j:
        lambda_t = weight[c_t]
        denominator += lambda_t

        for c_s in c_cap_j_prime:
            if c_t == c_s:
                print('\tmatch: %s, %s' % (c_t, c_s))
                numerator += lambda_t
    
    return numerator / denominator

def pair_scores(j, j_prime, weight):
    if(len(j)==0) or (len(j_prime)==0):
        curr_score = 0
    elif j_prime == j:
        curr_score = 1
    else:
        curr_score = label_score(j, j_prime, weight)
    
    print('\tj=%s, j_prime=%s, curr_score=%s' % (j, j_prime, curr_score))
    return float(curr_score)

# Debugging

* df = rec items
* df1 = gt items

In [46]:
# Debugging

weighing='equal'
col='hMatch-1'

def lst_scores(line, weight=weighing_scheme[weighing]):
    j = line['label_summary']
    j_primes = line['l2']
    pair_score = 0
    
    for j_prime in j_primes:
        print('j=%s, j_prime=%s' % (j, j_prime))
        s = pair_scores(j, j_prime, weight)
        if s > pair_score:
            pair_score = s
    print(line["choice"], j, j_prime, pair_score)
    return pair_score

test_df = df[(df["user"]==6) & (df["qn"]==2)].sort_values(by=["qn","user"])
test_df[col] = test_df.apply(lst_scores, axis=1)
# test_df1 = df1[(df1["user"]==6) & (df1["qn"]==2)].sort_values(by=["qn","user"])
# test_df1[col] = test_df1.apply(lst_scores, axis=1)
test_df

j=[1103, 1147, 1195], j_prime=[1103, 1147, 1156]
	match: 1103, 1103
	match: 1147, 1147
	j=[1103, 1147, 1195], j_prime=[1103, 1147, 1156], curr_score=0.6666666666666666
j=[1103, 1147, 1195], j_prime=[962, 984, 989]
	j=[1103, 1147, 1195], j_prime=[962, 984, 989], curr_score=0.0
Pasta - Cooked Pasta [1103, 1147, 1195] [962, 984, 989] 0.6666666666666666
j=[762, 833, 834], j_prime=[1103, 1147, 1156]
	j=[762, 833, 834], j_prime=[1103, 1147, 1156], curr_score=0.0
j=[762, 833, 834], j_prime=[962, 984, 989]
	j=[762, 833, 834], j_prime=[962, 984, 989], curr_score=0.0
Chicken Gumbo [762, 833, 834] [962, 984, 989] 0
j=[762, 833, 839], j_prime=[1103, 1147, 1156]
	j=[762, 833, 839], j_prime=[1103, 1147, 1156], curr_score=0.0
j=[762, 833, 839], j_prime=[962, 984, 989]
	j=[762, 833, 839], j_prime=[962, 984, 989], curr_score=0.0
Momo - Turkey Momo (Rwi) [762, 833, 839] [962, 984, 989] 0
j=[661, 689, 693, 694], j_prime=[1103, 1147, 1156]
	j=[661, 689, 693, 694], j_prime=[1103, 1147, 1156], curr_score=0.

,user,qn,choice,rating,item,gt,label,label_summary,l2,hMatch-1
260,6,2,Pasta - Cooked Pasta,3,pasta - cooked pasta,"[bubba burger - burger, fruits - salade fruits]","[staple, staple__wheat, staple__wheat__pasta]","[1103, 1147, 1195]","[[1103, 1147, 1156], [962, 984, 989]]",0.666667
261,6,2,Chicken Gumbo,4,chicken gumbo,"[bubba burger - burger, fruits - salade fruits]","[meat, meat__poultry, meat__poultry__chicken]","[762, 833, 834]","[[1103, 1147, 1156], [962, 984, 989]]",0.000000
262,6,2,Momo - Turkey Momo (Rwi),3,momo - turkey momo,"[bubba burger - burger, fruits - salade fruits]","[meat, meat__poultry, meat__poultry__turkey]","[762, 833, 839]","[[1103, 1147, 1156], [962, 984, 989]]",0.000000
263,6,2,Dry Fruits - Cantaloupe Dry,4,dry fruits - cantaloupe dry,"[bubba burger - burger, fruits - salade fruits]","[fruit, fruit__tropical, fruit__tropical__cant...","[661, 689, 693, 694]","[[1103, 1147, 1156], [962, 984, 989]]",0.000000
264,6,2,Bite Meals - Bite Meals Chicken Marsala,3,bite meals - bite meals chicken marsala,"[bubba burger - burger, fruits - salade fruits]","[meat, meat__poultry, meat__poultry__chicken]","[762, 833, 834]","[[1103, 1147, 1156], [962, 984, 989]]",0.000000
265,6,2,Noodles - Penne Noodles,4,noodles - penne noodles,"[bubba burger - burger, fruits - salade fruits]","[staple, staple__wheat, staple__wheat__noodle,...","[1103, 1147, 1189, 1196]","[[1103, 1147, 1156], [962, 984, 989]]",0.500000
266,6,2,Cheerios - Original Cheerios,5,cheerios - original cheerios,"[bubba burger - burger, fruits - salade fruits]","[beverage, beverage__alcohol, beverage__alcoho...","[9, 10, 35, 203, 204, 206]","[[1103, 1147, 1156], [962, 984, 989]]",0.000000
267,6,2,Milk - 2% White Milk,3,milk - 2% white milk,"[bubba burger - burger, fruits - salade fruits]","[egg_dairy, egg_dairy__dairy_product, egg_dair...","[526, 527, 542]","[[1103, 1147, 1156], [962, 984, 989]]",0.000000
268,6,2,Nonfat yogurt - Yogurt,4,nonfat yogurt - yogurt,"[bubba burger - burger, fruits - salade fruits]","[egg_dairy, egg_dairy__dairy_product, egg_dair...","[526, 527, 554]","[[1103, 1147, 1156], [962, 984, 989]]",0.000000
269,6,2,Banana,5,banana,"[bubba burger - burger, fruits - salade fruits]","[fruit, fruit__tropical, fruit__tropical__banana]","[661, 689, 692]","[[1103, 1147, 1156], [962, 984, 989]]",0.000000


In [199]:
# test_df1

In [200]:
weighing='equal'
col = 'hMatch-1'

def lst_scores(line, weight=weighing_scheme[weighing]):
    j = line['label_summary']
    j_primes = line['l2']
    pair_score = 0
    
    for j_prime in j_primes:
        s = pair_scores(j, j_prime, weight)
        if s > pair_score:
            pair_score = s
    return pair_score

import time
start = time.time()
df[col] = df.apply(lst_scores, axis=1)
df1[col] = df1.apply(lst_scores, axis=1)
end = time.time()
print(end - start)

0.11266326904296875


In [201]:
weighing='124'
col = 'hMatch-2'

def lst_scores(line, weight=weighing_scheme[weighing]):
    j = line['label_summary']
    j_primes = line['l2']
    pair_score = 0
    
    for j_prime in j_primes:
        s = pair_scores(j, j_prime, weight)
        if s > pair_score:
            pair_score = s
    return pair_score

import time
start = time.time()
df[col] = df.apply(lst_scores, axis=1)
df1[col] = df1.apply(lst_scores, axis=1)
end = time.time()
print(end - start)

0.17755985260009766


In [202]:
weighing='freq'
col = 'hMatch-freq'

def lst_scores(line, weight=weighing_scheme[weighing]):
    j = line['label_summary']
    j_primes = line['l2']
    pair_score = 0
    
    for j_prime in j_primes:
        s = pair_scores(j, j_prime, weight)
        if s > pair_score:
            pair_score = s
    return pair_score

import time
start = time.time()
df[col] = df.apply(lst_scores, axis=1)
df1[col] = df1.apply(lst_scores, axis=1)
end = time.time()
print(end - start)

0.11466121673583984


In [203]:
# precision
cols = ['user', 'qn', 'choice', 'rating', 'hMatch-1', 'hMatch-2', 'hMatch-freq']
d = df[cols]
d.columns = ['user', 'qn', 'rec_item', 'rating', 'hP-1', 'hP-2', 'hP-freq']
d.to_csv(preference_rating_scores['hP'], index=False)

In [204]:
# recall
cols = ['user', 'qn', 'item_1', 'hMatch-1', 'hMatch-2', 'hMatch-freq']
d = df1[cols]
d.columns = ['user', 'qn', 'gt_item', 'hR-1', 'hR-2', 'hR-freq']
d.to_csv(preference_rating_scores['hR'], index=False)